In [2]:
from keras.datasets.mnist import load_data
import numpy as np
import matplotlib.pyplot as plt
import pylab as p

In [3]:
(train_digits, train_labels), (test_digits, test_labels) = load_data()

11493376/11490434 [==============================] - 2s 0us/step


In [4]:
image_height = train_digits.shape[1]  
image_width = train_digits.shape[2]
num_channels = 1

In [5]:
train_data = np.reshape(train_digits, (train_digits.shape[0], image_height, image_width, num_channels))
test_data = np.reshape(test_digits, (test_digits.shape[0],image_height, image_width, num_channels))

In [6]:
train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

In [7]:
from keras.utils import to_categorical
num_classes = 10
train_labels_cat = to_categorical(train_labels,num_classes)
test_labels_cat = to_categorical(test_labels,num_classes)
train_labels_cat.shape, test_labels_cat.shape

((60000, 10), (10000, 10))

In [8]:
val_perc = 0.10
val_count = int(val_perc * len(train_data))

In [9]:
val_data = train_data[:val_count,:]
val_labels_cat = train_labels_cat[:val_count,:]

In [10]:
train_data2 = train_data[val_count:,:]
train_labels_cat2 = train_labels_cat[val_count:,:]